In [36]:
from skimage import io, transform, color
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
from dataLoader import OrganoidDataset
from torch.utils import data
import numpy as np
import sys
import pandas as pd
from imageio import imread
from PIL import Image
import os

from dataLoader import OrganoidDataset
#from conv_model import SimpleConvNet
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
figure_path = '../milestoneReport/figures/'
path = '../data/'
label_path = '../data/'

In [38]:
wellA1 = pd.read_csv('../data/well_summary_A1_e0891BSA_all.csv')
wellA2 = pd.read_csv(label_path+'well_summary_A2.csv')
wellA3 = pd.read_csv(label_path+'well_summary_A3.csv')
wellA4 = pd.read_csv(label_path+'well_summary_A4.csv')
wellC1 = pd.read_csv(label_path+'well_summary_C1.csv')

In [39]:
wellA1['condition'] = 'A1'
wellA2['condition'] = 'A2'
wellA3['condition'] = 'A3'
wellA4['condition'] = 'A4'
wellC1['condition'] = 'C1'

In [40]:
wellA1.shape, wellA2.shape,wellA3.shape,wellA4.shape,wellC1.shape

((67200, 7), (50400, 7), (50263, 7), (33600, 7), (44800, 7))

In [6]:
wellA1.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,125.0,7830,512,418,A1
1,1,0,125.0,8265,3044,3008,A1


In [7]:
wellA2.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,130.0,32396,11222,10702,A2
1,1,0,131.0,9500,43,0,A2


In [8]:
wellC1.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,117.0,7224,139,141,C1
1,1,0,118.0,7308,422,388,C1


In [9]:
wellA1.groupby(by='day')['well id'].count()

day
0     4800
1     4800
2     4800
3     4800
4     4800
5     4800
6     4800
7     4800
8     4800
9     4800
10    4800
11    4800
12    4800
13    4800
Name: well id, dtype: int64

In [10]:
wellA2.groupby(by='day')['well id'].count()

day
0     3600
1     3600
2     3600
3     3600
4     3600
5     3600
6     3600
7     3600
8     3600
9     3600
10    3600
11    3600
12    3600
13    3600
Name: well id, dtype: int64

In [11]:
wellA3.groupby(by='day')['well id'].count()

day
3    3600
5    3600
6    3596
Name: well id, dtype: int64

In [12]:
wellC1.groupby(by='day')['well id'].count()

day
0     3200
1     3200
2     3200
3     3200
4     3200
5     3200
6     3200
7     3200
8     3200
9     3200
10    3200
11    3200
12    3200
13    3200
Name: well id, dtype: int64

In [13]:
wellA2.groupby(by='day')['hyst2 area'].sum()

day
0     11118030
1      9515283
2      9699242
3     10188886
4      9387332
5     11237400
6     10763021
7     12244466
8     11280029
9     11625314
10    12835175
11    13009691
12    13693762
13    13596488
Name: hyst2 area, dtype: int64

In [14]:
wellC1.groupby(by='day')['hyst2 area'].sum()

day
0     1259622
1      928593
2      929989
3     1128199
4     1661649
5     2061019
6     1971614
7     1753965
8     1991609
9     2784164
10    2098197
11    2164090
12    2155446
13    2357182
Name: hyst2 area, dtype: int64

In [41]:
label = pd.concat([wellA1,wellA2,wellA3,wellA4,wellC1],sort=False)
new_columns = ['well_id','day','median_pixel_intensity','mw_area_shape','hyst1_area','hyst2_area','condition']
label.columns = new_columns

In [42]:
label.head(2)

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area,condition
0,0,0,125.0,7830,512,418,A1
1,1,0,125.0,8265,3044,3008,A1


In [43]:
def get_image_name(well_id,day_id):
    image_name = 'well'+str(well_id).zfill(4)+'_day'+str(day_id).zfill(2)+'_well.png'
    return image_name
def get_well_label(well_id):
    return str(well_id).zfill(4)

In [44]:
label['image_name'] = label[['well_id','day']].apply(lambda x: get_image_name(*x),axis=1)
label['image_name'] = 'well_'+label['condition']+'/'+label['image_name']
label['well_label'] = label['well_id'].apply(lambda x: get_well_label(x))
label['has_cell'] = np.int64(label['hyst2_area'] > 0)

In [45]:
label.head(3)

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area,condition,image_name,well_label,has_cell
0,0,0,125.0,7830,512,418,A1,well_A1/well0000_day00_well.png,0000,1
1,1,0,125.0,8265,3044,3008,A1,well_A1/well0001_day00_well.png,0001,1
2,2,0,127.0,8280,30,0,A1,well_A1/well0002_day00_well.png,0002,0


In [46]:
label.shape

(246263, 10)

In [47]:
corrected_labels = label.query("mw_area_shape < 32000")

In [48]:
corrected_labels.shape

(234814, 10)

In [49]:
def get_day_description(corrected_labels,day):
    df = corrected_labels.query("day == %d"%day)[['condition','well_id','day','well_label','image_name','has_cell']]
    day_suffix = '_' + str(day)
    df = df.rename(columns={'day':'day'+day_suffix, 'has_cell':'has_cell'+day_suffix,'image_name':'image_name'+day_suffix})
    return df

In [50]:
# day2wells = get_day_description(corrected_labels,2)
# day8wells = get_day_description(corrected_labels,8)

In [51]:
# day2wells.shape,day8wells.shape

In [52]:
# day2wells.merge(day8wells,on=['well_id','well_label','condition']).shape

In [53]:
prev = get_day_description(corrected_labels,0)
for day in range(1,14):
    #print(day)
    curr = get_day_description(corrected_labels,day)
    #print(curr.shape[0])
    prev = prev.merge(curr,on=['well_id','well_label','condition'])

In [54]:
prev.shape

(12621, 45)

In [55]:
merged_days = prev.copy()

In [56]:
#shuffle dataframe
merged_days = merged_days.sample(frac=1).reset_index(drop=True)

In [57]:
train,validation,test = np.split(merged_days, [int(.8 * len(merged_days)), int(.9 * len(merged_days))])

In [58]:
train.shape,validation.shape,test.shape

((10096, 45), (1262, 45), (1263, 45))

In [59]:
train.head(4)

,condition,well_id,day_0,well_label,image_name_0,has_cell_0,day_1,image_name_1,has_cell_1,day_2,...,has_cell_10,day_11,image_name_11,has_cell_11,day_12,image_name_12,has_cell_12,day_13,image_name_13,has_cell_13
0,A3,1381,0,1381,well_A3/well1381_day00_well.png,0,1,well_A3/well1381_day01_well.png,0,2,...,1,11,well_A3/well1381_day11_well.png,1,12,well_A3/well1381_day12_well.png,1,13,well_A3/well1381_day13_well.png,1
1,A1,2709,0,2709,well_A1/well2709_day00_well.png,1,1,well_A1/well2709_day01_well.png,0,2,...,0,11,well_A1/well2709_day11_well.png,0,12,well_A1/well2709_day12_well.png,0,13,well_A1/well2709_day13_well.png,0
2,A1,387,0,0387,well_A1/well0387_day00_well.png,0,1,well_A1/well0387_day01_well.png,0,2,...,0,11,well_A1/well0387_day11_well.png,0,12,well_A1/well0387_day12_well.png,0,13,well_A1/well0387_day13_well.png,1
3,A2,1750,0,1750,well_A2/well1750_day00_well.png,1,1,well_A2/well1750_day01_well.png,1,2,...,1,11,well_A2/well1750_day11_well.png,1,12,well_A2/well1750_day12_well.png,1,13,well_A2/well1750_day13_well.png,1


In [68]:
prev.to_csv("../data_description/A1_A2_A3_A4_C1_filtered_all_v2.csv")
train.to_csv("../data_description/A1_A2_A3_A4_C1_filtered_train_v2.csv")
validation.to_csv("../data_description/A1_A2_A3_A4_C1_filtered_validation_v2.csv")
test.to_csv("../data_description/A1_A2_A3_A4_C1_filtered_test_v2.csv")

# normalization

In [61]:
# def update_mean(x, x_bar_prev, n):
#   return x_bar_prev + (x - x_bar_prev)/n

# def update_sum_square_diff(x, ssd_prev, x_bar_prev, x_bar_curr):
#   return ssd_prev + (x - x_bar_prev)*(x - x_bar_curr)

In [62]:
# def get_mean_std(image_names,path = label_path):
#     #image_names = merged_days['image_name_1']
#     x_bar_curr = 0.0
#     x_bar_prev = 0.0
#     ssd_curr = 0.0
#     n = 0
#     for index in range(len(image_names)):
#         img_name = image_names[index]
#         img_loc = os.path.join(path, img_name)
#       # skimage.io.imread returns a numpy array
#         image = io.imread(img_loc)
#       # convert to grey scale
#         image = color.rgb2gray(image)
#         for i, x in np.ndenumerate(image):
#             n = n + 1
#             x_bar_prev = x_bar_curr
#             x_bar_curr = update_mean(x, x_bar_prev, n)
#             ssd_curr = update_sum_square_diff(x, ssd_curr, x_bar_prev, x_bar_curr)
#     return x_bar_curr, np.sqrt(ssd_curr/n)

In [64]:
def get_mean_std(image_names,path = label_path):
    all_images = []
    for index in range(len(image_names)):
        img_name = image_names[index]
        img_loc = os.path.join(path, img_name)
      # skimage.io.imread returns a numpy array
        image = io.imread(img_loc)
      # convert to grey scale
        image = color.rgb2gray(image)
        all_images.append(image)
    all_images = np.array(all_images)
    mean = np.mean(all_images)
    std = np.std(all_images)
    return mean, std

In [25]:
day2_mean, day2_std = get_mean_std(merged_days['image_name_2'])

In [26]:
day2_mean, day2_std

(0.4942364496332139, 0.16092630656380771)

In [63]:
train.shape

(10096, 45)

In [65]:
def get_mean_std_for_all_days(df = train, max_day_range=14):
    mean_sd_dict = {}
    for day in range(max_day_range):
        mean, std = get_mean_std(df['image_name_%d'%day])
        mean_sd_dict[day] = [mean, std]
    return mean_sd_dict

In [67]:
mean_sd_dict = get_mean_std_for_all_days()

In [31]:
mean_sd_dict

{0: [0.5069736899804134, 0.16161648095626624],
 1: [0.4983455891095547, 0.1620543794576955],
 2: [0.49439774802337344, 0.16087996922691195],
 3: [0.5036597384500131, 0.16115670884655703],
 4: [0.49723225904138163, 0.1613796328736843],
 5: [0.5013496452715945, 0.1605951051365687],
 6: [0.5015181095760308, 0.15958463728030045],
 7: [0.5141906038585404, 0.16129666771679602],
 8: [0.5177020917650417, 0.15714445907773483],
 9: [0.49718508281373336, 0.16299933786858656],
 10: [0.4962874322121036, 0.15880961979223254],
 11: [0.5179651916522097, 0.1604524823191784],
 12: [0.4987612682320938, 0.15931988930159605],
 13: [0.5045039581447099, 0.15828551745396297]}

In [33]:
mean_sd_df = pd.DataFrame.from_dict(mean_sd_dict,orient='index',columns=['mean','std'])

In [35]:
mean_sd_df

,mean,std
0,0.506974,0.161616
1,0.498346,0.162054
2,0.494398,0.160880
3,0.503660,0.161157
4,0.497232,0.161380
5,0.501350,0.160595
6,0.501518,0.159585
7,0.514191,0.161297
8,0.517702,0.157144
9,0.497185,0.162999


In [34]:
mean_sd_df.to_csv("../data_description/A1_A2_C1_mean_std.csv")